In [1]:
import matplotlib.pylab as plt
import pickle
import numpy as np
import os

In [2]:
import pandas as pd

In [3]:
import ising

In [4]:
plt.style.use('default')
plt.rc('figure', figsize = (15,8))
plt.rc('axes', labelsize=24)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

In [5]:
sizes = [ 2, 4, 8, 16]
mcsteps =[5e5]

In [ ]:
ising.main(sizes=sizes, mcsteps=mcsteps, TF = 10)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
from itertools import product, cycle

In [ ]:
results = {s: {} for s in sizes}
for size, resultsize in results.items():
    resultsize.update({nsteps: {} for nsteps in mcsteps})
for size, steps in product(sizes, mcsteps):
    results[size][steps] = pd.read_csv(f'results_{size}x{size}_{steps}.dat', header=0)

In [ ]:
markersymbols = ['o','s','d','^', 'P', 'X', 'h', '*']
linecolors = ['blue', 'red', 'orange', 'steelblue', 'purple', 'maroon']

In [ ]:
import matplotlib.lines as mlines

In [ ]:
def plot_convergence_size_steps(results, observable = 'Emean',
                               symbol = 'E'):
    fig, ax = plt.subplots()
    for j, (size, resultsize) in enumerate(results.items()):
        for i, (nsteps, resultsizestep) in enumerate(resultsize.items()):
            ax.plot(
                resultsizestep['T'], 
                resultsizestep[observable],
                '-', marker=markersymbols[i],markeredgecolor='k',markersize=8, color=linecolors[j])
    ax.set_ylabel(fr'$\langle {symbol} \rangle$ / per spin')
    ax.set_xlabel('T')
    sizes_handles = [mlines.Line2D([],[],linestyle='-', color = linecolors[i]) for i, size in enumerate(results.keys())]
    sizes_labels = [f'n={size}' for i, size in enumerate(results.keys())]
    nsteps_handles = [mlines.Line2D([],[],marker=markersymbols[i], color = 'k') for i, mcstep in enumerate(mcsteps)]
    nsteps_labels = [f'{n:.1e} mcsteps' for n in mcsteps]
    ax.legend(sizes_handles+nsteps_handles, sizes_labels+nsteps_labels)
    return fig,ax

In [ ]:
fig, ax = plot_convergence_size_steps(results, observable='Emean')
#ax.set_ylim([-2.1, 0.2])

In [ ]:
plot_convergence_size_steps(results, 'MabsMEAN', symbol='|M|')

In [ ]:
plot_convergence_size_steps(results, observable='CV', symbol='C_v')

Hay dos tipos de susceptibilidad:
$$ \chi = \dfrac{\langle M^2 \rangle - \langle M \rangle ^2}{k_B T} $$
$$ \chi '  = \dfrac{\langle M^2 \rangle - \langle |M| \rangle ^2}{k_B T} $$

for size, thisresult in results.items():
    for steps, result in thisresult.items():
        result['X'] = (result['M2acum']/size**2 - (result['Mmean'])**2)/result['T']

In [ ]:
plot_convergence_size_steps(results, observable='X', symbol = '\chi')

In [ ]:
plot_convergence_size_steps(results, observable='Xp', symbol = '\chi \' ')

# Comparación para solución teórica 2x2 

In [ ]:
def compare_2x2_teorico(observable, symbol):
    plt.plot (results['teorico']['T'],results['teorico'][observable], 'k', lw = 5, label='teórico 2x2')
    for (size, result), marker in zip(results[2].items(), markersymbols[:len(results[2])]):
        plt.plot (result['T'], result[observable], marker+'--r', markeredgecolor='k', label=f'{size}')
    plt.legend()
    plt.ylabel(fr'$\langle {symbol} \rangle$')
    plt.xlabel (r'$\dfrac{T}{T_c}$')

In [ ]:
results['teorico'] = {} 

In [ ]:
results['teorico']['T'] =results[2][1e4]['T']

In [ ]:
beta = 1/results['teorico']['T']

In [ ]:
teoricos = ising.SolucionTeorica2x2(beta, 1)

In [ ]:
results['teorico'].update({'Emean': teoricos.emean()})

In [ ]:
results['teorico'].update({'E2mean': teoricos.e2mean()})

In [ ]:
results['teorico'].update({'MabsMEAN': teoricos.mmean()})

In [ ]:
results['teorico'].update({'M2acum': teoricos.m2mean()})

In [ ]:
results['teorico']['CV'] = (results['teorico']['E2mean']/4 - results['teorico']['Emean']**2)*beta**2*4

In [ ]:
results['teorico']['Xp'] = (results['teorico']['M2acum']/4 - results['teorico']['MabsMEAN']**2)*beta*4

In [ ]:
compare_2x2_teorico('Emean', 'E')

In [ ]:
compare_2x2_teorico('E2mean', 'E^2')

In [ ]:
compare_2x2_teorico('MabsMEAN', '|M|')

In [ ]:
compare_2x2_teorico('M2acum', 'M^2')

In [ ]:
compare_2x2_teorico('CV', 'C_v')

In [ ]:
compare_2x2_teorico('Xp', '\chi')